In [ ]:
!pip install scikit-surprise

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip freeze --local > /content/gdrive/My\ Drive/colab_installed.txt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
import pandas as pd
import os
import numpy as np
from collections import defaultdict

In [ ]:
ratings_file = "/content/gdrive/MyDrive/users.csv"
df = pd.read_csv(ratings_file)


In [ ]:
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0, 100))

# data = Dataset.load_from_file(file_path, reader=reader)
data = Dataset.load_from_df(df[['user_id', 'artist_name', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.2)

In [ ]:
# kf = KFold(n_splits=3)

algo = SVD()


algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 62.1644


62.16439486002085

In [ ]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [ ]:
# testset = trainset.build_anti_testset()
# testset = df.head(100)
# predictions = algo.test(testset)

# top_n = get_top_n(predictions, n=10)
# print(top_n)
test = []
for i in range(1,10000):
  tup = (df.loc[i]['user_id'], df.loc[i]['artist_name'], df.loc[i]['rating'])
  test.append(tup)
predictions = algo.test(test)

top_n = get_top_n(predictions, n=10)

In [ ]:
new = pd.DataFrame.from_dict(top_n, orient='index')
new.to_csv('/content/gdrive/MyDrive/SVD_2.csv')

In [ ]:
ratings_file = "/content/gdrive/MyDrive/users.csv"
df = pd.read_csv(ratings_file)
